In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from datetime import datetime
#%matplotlib inline
#import seaborn as sns
from sklearn.model_selection import train_test_split
from time import time
from sklearn.metrics import f1_score
from sklearn.preprocessing import scale
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
random_state=401
n_jobs = 6

In [3]:
cd Data

/Users/Shalu/Dropbox/Coursework/DS/KLC/Data


In [4]:
#dfc = pd.read_pickle('dfclean104.p')
#dfc = pd.read_pickle('dfc_fwg_emp_1024.p')
#dfc = pd.read_pickle('dfc_fwg_emp_1024.p')
#dfc = pd.read_pickle('deltempZipFuzzyMerge.p')
dfc = pd.read_pickle('df_log_reg.p')

In [5]:
dfc.shape

(232487, 35)

In [6]:
#df_reduce = dfc.copy()
df_reduce = dfc.copy()
del dfc

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import scale
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

In [8]:
def my_scorer(clf, X, y):
    y_pred_prob = clf.predict_proba(X)[:,1]
    return roc_auc_score(y, y_pred_prob)

In [9]:
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=random_state)
scoring_function = my_scorer

## Run Random Forest On Both Features

In [26]:
features = ['loan_amnt', 'term', 'int_rate', 'home_ownership', u'annual_inc', 'verification_status', 'purpose', 
            'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 
            'revol_util', 'initial_list_status','collections_12_mths_ex_med', 'mort_acc',
            'fico_avg', 'emp_length_cat', 'fw_emp_title_gt0005', 'zip_gt001'
           ]

target = ['loan_label']

X=df_reduce[features]
print X.columns
X = pd.get_dummies(X)
col = X.columns
X = scale(X)
y=df_reduce[target].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=.3, random_state=random_state)

Index([u'loan_amnt', u'term', u'int_rate', u'home_ownership', u'annual_inc', u'verification_status', u'purpose', u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc', u'fico_avg', u'emp_length_cat', u'fw_emp_title_gt0005', u'zip_gt001'], dtype='object')


In [42]:
from sklearn.ensemble import RandomForestClassifier
g_rnf = RandomForestClassifier(random_state=random_state)
n_est_range = [1200]
max_features_range = [.3, .6] 
min_samples_leaf_range=[100] 
parameters = dict(n_estimators=n_est_range, max_features=max_features_range, min_samples_leaf=min_samples_leaf_range)

grid_rnf = GridSearchCV(estimator=g_rnf, param_grid=parameters, cv=sss, n_jobs=n_jobs, scoring=my_scorer, refit=True)

In [18]:
import caffeine
start = time()
grid_rnf.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
print grid_rnf.best_score_
print grid_rnf.best_estimator_
print grid_rnf.cv_results_
caffeine.off()

8349.7935 seconds
0.690566027524
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=100,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1200, n_jobs=1, oob_score=False, random_state=401,
            verbose=0, warm_start=False)


In [39]:
grid_rnf.cv_results_

{'mean_fit_time': array([ 2075.98488601,  3756.17199834]),
 'mean_score_time': array([ 7.96730089,  6.91097331]),
 'mean_test_score': array([ 0.69056603,  0.68981213]),
 'mean_train_score': array([ 0.75756852,  0.76765895]),
 'param_max_features': masked_array(data = [0.3 0.6],
              mask = [False False],
        fill_value = ?),
 'param_min_samples_leaf': masked_array(data = [100 100],
              mask = [False False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [1200 1200],
              mask = [False False],
        fill_value = ?),
 'params': ({'max_features': 0.3,
   'min_samples_leaf': 100,
   'n_estimators': 1200},
  {'max_features': 0.6, 'min_samples_leaf': 100, 'n_estimators': 1200}),
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([ 0.68455124,  0.68409717]),
 'split0_train_score': array([ 0.75837331,  0.76825352]),
 'split1_test_score': array([ 0.69426693,  0.6936473 ]),
 'split1_train_score': array([ 0.75692536, 

In [27]:
y_pred_prob = grid_rnf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

0.69365630684193391

## Run on emp title only

In [10]:
features = ['loan_amnt', 'term', 'int_rate', 'home_ownership', u'annual_inc', 'verification_status', 'purpose', 
            'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 
            'revol_util', 'initial_list_status','collections_12_mths_ex_med', 'mort_acc',
            'fico_avg', 'emp_length_cat', 'fw_emp_title_gt0005',# 'zip_gt001'
           ]

target = ['loan_label']

X=df_reduce[features]
print X.columns
X = pd.get_dummies(X)
col = X.columns
X = scale(X)
y=df_reduce[target].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=.3, random_state=random_state)



Index([u'loan_amnt', u'term', u'int_rate', u'home_ownership', u'annual_inc', u'verification_status', u'purpose', u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc', u'fico_avg', u'emp_length_cat', u'fw_emp_title_gt0005'], dtype='object')


In [44]:
import caffeine
start = time()
grid_rnf.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
print grid_rnf.best_score_
print grid_rnf.best_estimator_
print grid_rnf.cv_results_
caffeine.off()
y_pred_prob = grid_rnf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

5490.2047 seconds
0.690256003743
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=100,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1200, n_jobs=1, oob_score=False, random_state=401,
            verbose=0, warm_start=False)
{'std_train_score': array([ 0.00048728,  0.00040339]), 'rank_test_score': array([1, 2], dtype=int32), 'mean_score_time': array([ 7.0251046,  6.1480449]), 'std_test_score': array([ 0.00422684,  0.00415545]), 'split1_train_score': array([ 0.75677305,  0.76681434]), 'split0_test_score': array([ 0.68435519,  0.68405134]), 'mean_test_score': array([ 0.690256  ,  0.68976078]), 'split2_train_score': array([ 0.75728826,  0.76716743]), 'split0_train_score': array([ 0.75796307,  0.7677901 ]), 'param_min_samples_leaf': masked_array(data = [100 100],
             mask = [False False],
    

0.69344546970238596

In [11]:
from sklearn.ensemble import RandomForestClassifier
g_rnf = RandomForestClassifier(random_state=random_state, n_estimators=1200, 
                               max_features=.3, min_samples_leaf=100, n_jobs=1)

In [12]:
start = time()
g_rnf.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
y_pred_prob = g_rnf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

2775.0623 seconds


0.69344546970238596

In [14]:
import pickle

with open('g_rnf.p', 'w') as f_w:
    pickle.dump(g_rnf, f_w)

#with open('g_rnf.p', 'r') as f_r:
#    dt_model_load = pickle.load(f_r)


In [32]:
t = pd.DataFrame(zip(col, g_rnf.feature_importances_), columns=['features','coef'])
t['coef']=abs(t.coef)

In [35]:
t.sort_values(by='coef', ascending=False).head(30)

,features,coef
1,int_rate,0.403545
3,dti,0.073820
2,annual_inc,0.066619
13,fico_avg,0.065220
9,revol_bal,0.056021
0,loan_amnt,0.053544
10,revol_util,0.046716
5,earliest_cr_line,0.036371
7,open_acc,0.027148
12,mort_acc,0.025582


In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
start = time()
rnf.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
y_pred_prob1 = rnf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob1)

341.8565 seconds


0.69183079882427734

### Parameter Tuning

In [65]:
n_est_range = [500, 1000, 1500, 2000, 3000]

In [46]:
scores = []
for n in n_est_range:
    start = time()  
    rnf = RandomForestClassifier(n_estimators=n, max_features=.3, min_samples_leaf=100, n_jobs=n_jobs, random_state=random_state)
    rnf.fit(X_train,y_train)
    y_pred_prob1 = rnf.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, y_pred_prob1)
    end = time()
    duration = (end - start)/60
    scores.append(dict(n_estimators=n, auc=score, duration=duration))
    
scores = pd.DataFrame(scores)

In [47]:
scores

,auc,duration,n_estimators
0,0.693253,2.648508,500
1,0.693360,5.383980,1000
2,0.693370,7.927020,1500
3,0.693489,16.850928,2000
4,0.693378,16.109152,3000


In [74]:
max_features_range = [.1, .2, .25, .3, .35, .4, .5] 

scores = []
for max_f in max_features_range:
    start = time()  
    rnf = RandomForestClassifier(n_estimators=2000, max_features=max_f, min_samples_leaf=100, n_jobs=n_jobs, random_state=random_state)
    rnf.fit(X_train,y_train)
    y_pred_prob1 = rnf.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, y_pred_prob1)
    end = time()
    duration = (end - start)/60
    scores.append(dict(max_features=max_f, auc=score, duration=duration))
    
scores = pd.DataFrame(scores)
print scores

        auc   duration  max_features
0  0.690995   3.899073          0.10
1  0.693189   7.611377          0.20
2  0.693366   9.392643          0.25
3  0.693467  10.378140          0.30
4  0.693200  11.596375          0.35
5  0.693106  13.051303          0.40
6  0.692884  16.203153          0.50


In [75]:
scores

,auc,duration,max_features
0,0.690995,3.899073,0.10
1,0.693189,7.611377,0.20
2,0.693366,9.392643,0.25
3,0.693467,10.378140,0.30
4,0.693200,11.596375,0.35
5,0.693106,13.051303,0.40
6,0.692884,16.203153,0.50


In [76]:
#n_est_range = [500,1000,1500,2000,3000]
#max_features_range = [.1, .3, .5, .7, .9] 
min_samples_leaf_range=[10, 50, 100, 300, 500, 1000, 3000]

scores = []
for min_samples in min_samples_leaf_range:
    start = time()  # let's time it, to see how long running a forest takes
    rnf = RandomForestClassifier(n_estimators=2000, max_features=.3, min_samples_leaf=100, n_jobs=n_jobs, random_state=random_state)
    rnf.fit(X_train,y_train)
    y_pred_prob1 = rnf.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, y_pred_prob1)
    end = time()
    duration = (end - start)/60
    scores.append(dict(min_samples_leaf=min_samples, auc=score, duration=duration))
    
scores = pd.DataFrame(scores)

In [77]:
print scores

        auc   duration  min_samples_leaf
0  0.693467   9.963278                10
1  0.693467   9.960127                50
2  0.693467   9.952173               100
3  0.693467   9.982697               300
4  0.693467   9.954201               500
5  0.693467   9.961517              1000
6  0.693467  10.206830              3000
